# Basic Loading of the Kernel

The Semantic Kernel SDK can be imported from the following nuget feed:

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.45.0"

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0 Microsoft.SemanticKernel.Yaml, 1.45.0

After adding the nuget package, you can instantiate the kernel:


In [3]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Inject your logger 
// see Microsoft.Extensions.Logging.ILogger @ https://learn.microsoft.com/dotnet/core/extensions/logging
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;

var builder = Kernel.CreateBuilder();
builder.Services.AddSingleton(myLoggerFactory); //Add logging!
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

Console.WriteLine ( await kernel.InvokePromptAsync("Tell me a joke about cats") );

Why did the cat sit on the computer?

Because it wanted to keep an eye on the mouse!


Now, let us try templatized prompts.

In [15]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

string promptTemplate = "Write funny tweet about {{$topic}}";

Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "people who run for fun" } } ) );
Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "seattle weather" } } ) );
Console.WriteLine ( await kernel.InvokePromptAsync(promptTemplate, new() { { "topic", "spicy noodels" } } ) );


"People who run for fun always tell me about their 'runner's high.' I usually nod along, but I'm pretty sure I get the same feeling by successfully finding a parking spot near the store entrance. 🏃‍♂️➡️🛒 #DifferentStrokes"
"Seattle's weather forecast: partly cloudy with a 100% chance of carrying an umbrella while optimistically wearing sunglasses. ☔️😎 #FourSeasonsInOneDay"
"Just ate some spicy noodles so hot they made my tongue file for divorce. It even got custody of the taste buds. #SizzlingSeparation 🌶️🍜"


In [ ]:
public class ResponsibleAIPromptFilter() : IPromptRenderFilter
    {        
        /// <summary>
        /// Method which is called asynchronously before prompt rendering.
        /// </summary>
        /// <param name="context">Instance of <see cref="PromptRenderContext"/> with prompt rendering details.</param>
        /// <param name="next">Delegate to the next filter in pipeline or prompt rendering operation itself. If it's not invoked, next filter or prompt rendering won't be invoked.</param>
        public async Task OnPromptRenderAsync(PromptRenderContext context, Func<PromptRenderContext, Task> next)
        {
            if (context.Arguments.ContainsName("card_number"))
            {
                context.Arguments["card_number"] = "**** **** **** ****";
            }

            await next(context);

            context.RenderedPrompt += " NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY."; // Add this line to filter out bias or bigotry
            context.RenderedPrompt += " NO HATE SPEECH OR HARASSMENT."; // Add this line to filter out hate speech or harassment
            context.RenderedPrompt += " NO VIOLENCE OR HARMFUL CONTENT."; // Add this line to filter out violence or harmful content
            context.RenderedPrompt += " NO ILLEGAL OR UNETHICAL CONTENT."; // Add this line to filter out illegal or unethical content            
        }
    }

##### And now, some yaml!

In [ ]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
#!import config/Settings.cs 


var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
builder.Services.AddSingleton<IPromptRenderFilter, ResponsibleAIPromptFilter>();
var kernel = builder.Build();
        
// Load prompt from resources
StreamReader reader = new("resources/prompts/GenerateExcuse.yaml", System.Text.Encoding.UTF8);
string generateExcuseYaml = reader.ReadToEnd();
reader.Close();

KernelFunction function = kernel.CreateFunctionFromPromptYaml(generateExcuseYaml);

// Invoke the prompt function and display the result
Console.WriteLine(await kernel.InvokeAsync(function, arguments: new()
{
    { "event", "Did not submit connect" },
    { "length", "1" },
}));

       

Excuse
A function that generates an excuse about an event
I'm sorry for not submitting the report, but a squirrel stole my laptop thinking it was a deluxe acorn, and I was too busy negotiating a trade agreement involving peanuts and a tiny helmet to get it back in time!
